In [8]:
import pandas as pd
from gensim.models import TfidfModel
from gensim.corpora import Dictionary
import numpy as np

In [30]:
arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")

/var/folders/cf/xcc222mj0yg3dzy7mljbx99w0000gn/T/ipykernel_99072/777748067.py:1: DtypeWarning: Columns (37,38) have mixed types. Specify dtype option on import or set low_memory=False.
  arcom_df = pd.read_csv("/Users/thibaultrolland/Projects/arcom/data/ARCOM.csv")


In [31]:
arcom_df.columns
arcom_df = arcom_df.dropna(subset=['visuel_clean'])
arcom_df = arcom_df[arcom_df['Secteur'] == 'AALIMENTATION']
arcom_df.reset_index(drop=True, inplace=True)

In [32]:
len(arcom_df)

7525

# tf-idf

In [33]:
text_series = arcom_df['visuel_clean'].apply(lambda x: x.split(' '))
labels_series = arcom_df['mots_clefs_themes'].apply(lambda x: [y.strip() for y in x.split(',')] )

print(f'text count: {len(text_series)}')
print(f'label count: {len(labels_series)}')

text count: 7525
label count: 7525


In [34]:
for i, v in pd.Series(
        np.concatenate(
            arcom_df['mots_clefs_themes'].map(lambda x: [x.strip() for x in x.split(',')])
        )
    ).value_counts().items():
    print(i + ' : ' + str(v))
    if v <= 100: break

GOURMANDISE : 3571
GOUT : 3062
TYPOGRAPHIE : 1511
TAG : 1434
ENFANT : 1403
BONNE HUMEUR : 1375
CHOIX : 1165
NOUVEAUTE AUTRE : 1130
SIGNE DE QUALITE AUTRE : 1099
JOIE : 1085
HUMOUR AUTRE : 1080
COMPLICITE : 1053
RECETTE : 1029
ANIMAL AUTRE : 979
CONVIVIALITE : 969
FAMILLE : 947
EFFET VISUEL : 941
INTERPELLATION : 896
SAVOIR-FAIRE AUTRE : 841
PLAISIR : 840
NATURE : 818
EXPERIENCE : 779
UNIVERS RURAL : 753
EFFET : 751
SIGNATURE SONORE : 745
MUSIQUE CELEBRE : 742
MUSIQUE AUTRE : 728
LIEN MEDIA : 719
ETONNEMENT : 715
COMPORTEMENT AUTRE : 707
COUPLE : 705
BRUITAGE AUTRE : 691
OBJET PERSONNIFIE : 671
COMPOSANT SPECIAL : 666
CONDITIONNEMENT : 660
ORIGINE : 659
INTERROGATION : 646
MEDIA INTERNET : 643
QUOTIDIEN : 642
AMITIE : 594
COULEUR : 585
CHANGEMENT DE SIGNATURE : 571
SENSATIONS AUTRE : 567
FESTIF : 560
UNIVERS PROFESSIONNEL : 548
SNACKING : 541
LIEN PATERNEL : 537
ANGLAIS : 529
GAMME : 528
SIGNATURE EXCLAMATIVE : 505
AMOUR : 489
NATUREL : 488
LIEN MATERNEL : 485
LUDIQUE : 472
PRATIQUE : 4

In [35]:
dictionary = Dictionary(text_series)
dictionary
corpus = [dictionary.doc2bow(text) for text in text_series]
model = TfidfModel(corpus)

In [36]:
target_label = 'NATURE'

target_texts = text_series[[target_label in labels for labels in labels_series.values]]
target_corpus = [dictionary.doc2bow(text) for text in target_texts]
target_tfidf = model[target_corpus]

In [37]:
threshold = 0.5
word_dict = {}
for sentence in target_tfidf:
    for word in sentence:
        if word[1] > threshold:
            if dictionary[word[0]] not in word_dict:
                word_dict[dictionary[word[0]]] = 1
            else:
                word_dict[dictionary[word[0]]] += 1

In [38]:
dict(sorted(word_dict.items(), key=lambda item: item[1], reverse=True))

{'façon': 20,
 'roquefort': 20,
 'sardine': 8,
 'lapin': 8,
 'marcher': 6,
 'imaginaire': 6,
 'pluie': 6,
 'jardin': 6,
 'soupe': 6,
 'country': 6,
 'framboise': 5,
 'gogh': 5,
 'soignon': 4,
 'tomme': 4,
 'fleur': 4,
 'cannelle': 4,
 'malvoyant': 4,
 'enceinte': 4,
 'normand': 3,
 'danette': 3,
 'noix': 3,
 'paître': 3,
 'epi': 3,
 'nourrir': 3,
 'camion': 3,
 'explicatif': 2,
 'louche': 2,
 'individuel': 2,
 'portion': 2,
 'kloboukoff': 2,
 'ermitage': 2,
 'plante': 2,
 'généreux': 2,
 'basque': 2,
 'thé': 2,
 'récit': 2,
 'attention': 2,
 'main': 2,
 'tissu': 2,
 'cream': 2,
 'infusion': 2,
 'enumération': 2,
 'nocciolata': 2,
 'brebis': 2,
 'louer': 2,
 'entrecouper': 2,
 'ciboulette': 2,
 'recyclable': 2,
 'sésame': 2,
 'racine': 2,
 'biostim': 2,
 'débuter': 2,
 'brocoli': 2,
 'ficello': 1,
 'japonais': 1,
 'moutarde': 1,
 'agur': 1,
 'norvege': 1,
 'cranberry': 1,
 'effilocher': 1,
 'muguet': 1,
 'fitnes': 1,
 'volailler': 1,
 'coque': 1,
 'sarrasin': 1,
 'bûche': 1,
 'chaume': 